In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats 
import plotly.express as px
import plotly.graph_objects as go
import nbformat
from sklearn.impute import SimpleImputer
from io import StringIO
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, chi2
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier



In [2]:
# Load the dataset (replace 'your_dataset.csv' with your actual dataset file)
df = pd.read_csv('Travel.csv')

In [3]:
# Display the first few rows of the dataset
print("First few rows of the dataset:")
df

First few rows of the dataset:


,CustomerID,ProdTaken,Age,TypeofContact,CityTier,DurationOfPitch,Occupation,Gender,NumberOfPersonVisiting,NumberOfFollowups,ProductPitched,PreferredPropertyStar,MaritalStatus,NumberOfTrips,Passport,PitchSatisfactionScore,OwnCar,NumberOfChildrenVisiting,Designation,MonthlyIncome
0,200000,1,41.0,Self Enquiry,3,6.0,Salaried,Female,3,3.0,Deluxe,3.0,Single,1.0,1,2,1,0.0,Manager,20993.0
1,200001,0,49.0,Company Invited,1,14.0,Salaried,Male,3,4.0,Deluxe,4.0,Divorced,2.0,0,3,1,2.0,Manager,20130.0
2,200002,1,37.0,Self Enquiry,1,8.0,Free Lancer,Male,3,4.0,Basic,3.0,Single,7.0,1,3,0,0.0,Executive,17090.0
3,200003,0,33.0,Company Invited,1,9.0,Salaried,Female,2,3.0,Basic,3.0,Divorced,2.0,1,5,1,1.0,Executive,17909.0
4,200004,0,NaN,Self Enquiry,1,8.0,Small Business,Male,2,3.0,Basic,4.0,Divorced,1.0,0,5,1,0.0,Executive,18468.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4883,204883,1,49.0,Self Enquiry,3,9.0,Small Business,Male,3,5.0,Deluxe,4.0,Unmarried,2.0,1,1,1,1.0,Manager,26576.0
4884,204884,1,28.0,Company Invited,1,31.0,Salaried,Male,4,5.0,Basic,3.0,Single,3.0,1,3,1,2.0,Executive,21212.0
4885,204885,1,52.0,Self Enquiry,3,17.0,Salaried,Female,4,4.0,Standard,4.0,Married,7.0,0,1,1,3.0,Senior Manager,31820.0
4886,204886,1,19.0,Self Enquiry,3,16.0,Small Business,Male,3,4.0,Basic,3.0,Single,3.0,0,5,0,2.0,Executive,20289.0


In [4]:
# Display the general information about the dataset
print("\nGeneral Information:")
print(df.info())


General Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4888 entries, 0 to 4887
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   CustomerID                4888 non-null   int64  
 1   ProdTaken                 4888 non-null   int64  
 2   Age                       4662 non-null   float64
 3   TypeofContact             4863 non-null   object 
 4   CityTier                  4888 non-null   int64  
 5   DurationOfPitch           4637 non-null   float64
 6   Occupation                4888 non-null   object 
 7   Gender                    4888 non-null   object 
 8   NumberOfPersonVisiting    4888 non-null   int64  
 9   NumberOfFollowups         4843 non-null   float64
 10  ProductPitched            4888 non-null   object 
 11  PreferredPropertyStar     4862 non-null   float64
 12  MaritalStatus             4888 non-null   object 
 13  NumberOfTrips             4748 non-null  

In [5]:
# Descriptive statistics for numerical columns
numerical_stats = df.describe()

# Descriptive statistics for categorical columns
categorical_stats = pd.DataFrame()
for col in df.select_dtypes(include=['object', 'category']):
    categorical_stats[col] = [df[col].nunique(),
                              df[col].value_counts().idxmax(),
                              df[col].value_counts().max() / len(df) * 100]

categorical_stats = categorical_stats.rename(columns={0: 'Unique Categories', 1: 'Most Common Category', 2: 'Percentage'})

# Print descriptive statistics
print("Descriptive Statistics for Numerical Columns:")
print(numerical_stats)
print("\nDescriptive Statistics for Categorical Columns:")
print(categorical_stats)


Descriptive Statistics for Numerical Columns:
          CustomerID    ProdTaken          Age     CityTier  DurationOfPitch  \
count    4888.000000  4888.000000  4662.000000  4888.000000      4637.000000   
mean   202443.500000     0.188216    37.622265     1.654255        15.490835   
std      1411.188388     0.390925     9.316387     0.916583         8.519643   
min    200000.000000     0.000000    18.000000     1.000000         5.000000   
25%    201221.750000     0.000000    31.000000     1.000000         9.000000   
50%    202443.500000     0.000000    36.000000     1.000000        13.000000   
75%    203665.250000     0.000000    44.000000     3.000000        20.000000   
max    204887.000000     1.000000    61.000000     3.000000       127.000000   

       NumberOfPersonVisiting  NumberOfFollowups  PreferredPropertyStar  \
count             4888.000000        4843.000000            4862.000000   
mean                 2.905074           3.708445               3.581037   
std     

In [6]:
# Check data types
print("\nData Types:")
print(df.dtypes)


Data Types:
CustomerID                    int64
ProdTaken                     int64
Age                         float64
TypeofContact                object
CityTier                      int64
DurationOfPitch             float64
Occupation                   object
Gender                       object
NumberOfPersonVisiting        int64
NumberOfFollowups           float64
ProductPitched               object
PreferredPropertyStar       float64
MaritalStatus                object
NumberOfTrips               float64
Passport                      int64
PitchSatisfactionScore        int64
OwnCar                        int64
NumberOfChildrenVisiting    float64
Designation                  object
MonthlyIncome               float64
dtype: object


In [7]:

# Assuming 'df' is your DataFrame containing the data

# Distribution of Age
fig1 = px.histogram(df, x='Age', nbins=10, title='Distribution of Age', marginal='box', hover_data=df.columns)
fig1.show()

# Pie Chart of Product Taken
fig2 = px.pie(df, names='ProdTaken', title='Distribution of Product Taken')
fig2.show()

# Boxplot of Monthly Income by Gender
fig3 = px.box(df, x='Gender', y='MonthlyIncome', title='Monthly Income by Gender', points='all')
fig3.show()

# Barplot of Product Pitched by Number of Followups
fig4 = px.bar(df, x='NumberOfFollowups', y='ProdTaken', color='ProductPitched', barmode='group',
              title='Product Pitched by Number of Followups')
fig4.show()

# Heatmap of correlation matrix
numeric_columns = df.select_dtypes(include=['number']).columns
correlation_matrix = df[numeric_columns].corr()

fig5 = go.Figure(data=go.Heatmap(
                   z=correlation_matrix.values,
                   x=correlation_matrix.columns,
                   y=correlation_matrix.columns,
                   colorscale='Viridis'))

fig5.update_layout(title='Correlation Matrix')
fig5.show()

# Scatter Plot of Age vs Monthly Income by Gender
fig6 = px.scatter(df, x='Age', y='MonthlyIncome', color='Gender',
                  title='Age vs Monthly Income by Gender',
                  hover_data=['Designation', 'CityTier'])
fig6.show()

# Pie Chart of Marital Status Distribution
fig7 = px.pie(df, names='MaritalStatus', title='Marital Status Distribution')
fig7.show()

# Sunburst Chart for Multi-level Hierarchical Data (CityTier -> Gender -> ProductPitched)
fig8 = px.sunburst(df, path=['CityTier', 'Gender', 'ProductPitched'],
                   values='CustomerID', title='Sunburst Chart of CityTier, Gender and Product Pitched')
fig8.show()

# Violin Plot of Monthly Income by Designation
fig9 = px.violin(df, x='Designation', y='MonthlyIncome',
                 title='Monthly Income by Designation',
                 box=True, points='all')
fig9.show()

# Grouped Bar Plot for Number of Trips vs Age by Product Taken
fig10_alternative = px.bar(df, x='Age', y='NumberOfTrips', color='ProdTaken',
                            title='Number of Trips vs Age by Product Taken',
                            barmode='group', hover_data=['CustomerID'])
fig10_alternative.show()

# Scatter Plot of Age vs Duration of Pitch colored by Gender
fig11_alternative = px.scatter(df, x='Age', y='DurationOfPitch', color='Gender',
                               title='Age vs Duration of Pitch by Gender',
                               hover_data=['CustomerID'])
fig11_alternative.show()

